In [1]:
#Install Tensorflow and MNIST dataset if not installed
!pip install tensorflow==2.7.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.6/489.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 KB 17.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2


In [2]:
!pip install openfl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.5/612.5 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet
tf.config.run_functions_eagerly(True)
tf.random.set_seed(0)
np.random.seed(0)

In [4]:
def test_intel_tensorflow():
    """
    Check if Intel version of TensorFlow is installed
    """
    import tensorflow as tf

    print("We are using Tensorflow version {}".format(tf.__version__))

    major_version = int(tf.__version__.split(".")[0])
    if major_version >= 2:
        from tensorflow.python.util import _pywrap_util_port
        print("Intel-optimizations (DNNL) enabled:",
              _pywrap_util_port.IsMklEnabled())
    else:
        print("Intel-optimizations (DNNL) enabled:")

test_intel_tensorflow()

We are using Tensorflow version 2.7.0
Intel-optimizations (DNNL) enabled: False


In [5]:
#Setup default workspace, logging, etc.
fx.init('keras_cnn_mnist')


Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── save
├── src
│   ├── keras_cnn.py
│   ├── mnist_utils.py
│   ├── tfmnist_inmemory.py
│   └── __init__.py
├── .workspace
├── plan
│   ├── data.yaml
│   ├── cols.yaml
│   ├── defaults
│   └── plan.yaml
├── cert
├── requirements.txt
├── logs
└── data

6 directories, 10 files
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request and Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing Certificate CSR
3   Create Certificate Chain

Done.
Creating AGGREGATOR certificate key pair with following settings: CN=e2498b5a2b64, SAN=DNS:e2498b5a2b64
  Writing AGGREGATOR certificate key pair to: /content/cert/server
The CSR Hash for file server/agg_e2498b5a2b64.csr = 132c3349340a47021b315df2af848900bcf214253ef9748492f5cfe89275479e470b9f71d4b1be0

In [6]:
#Import and process training, validation, and test images/labels

# Set the ratio of validation imgs, can't be 0.0
VALID_PERCENT = 0.3

(X_train, y_train), (X_test, y_test) = mnist.load_data()
split_on = int((1 - VALID_PERCENT) * len(X_train))

train_images = X_train[0:split_on,:,:]
train_labels = to_categorical(y_train)[0:split_on,:]

valid_images = X_train[split_on:,:,:]
valid_labels = to_categorical(y_train)[split_on:,:]

test_images = X_test
test_labels = to_categorical(y_test)

def preprocess(images):
    #Normalize
    images = (images / 255) - 0.5
    images = images.reshape(images.shape[0], -1)
#     images = np.expand_dims(images, axis=-1)
    return images

# Preprocess the images.
train_images = preprocess(train_images)
valid_images = preprocess(valid_images)
test_images = preprocess(test_images)

feature_shape = train_images.shape[1:]
classes = 10

class UnbalancedFederatedDataset(FederatedDataSet):
    def split(self, num_collaborators, shuffle=True, equally=False):
        train_idx = self.split_lognormal(self.y_train, num_collaborators)
        X_train = np.array([self.X_train[idx] for idx in train_idx])
        y_train = np.array([self.y_train[idx] for idx in train_idx])
        
        valid_idx = self.split_lognormal(self.y_valid, num_collaborators)
        X_valid = np.array([self.X_valid[idx] for idx in valid_idx])
        y_valid = np.array([self.y_valid[idx] for idx in valid_idx])
        
        return [
            FederatedDataSet(
                X_train[i],
                y_train[i],
                X_valid[i],
                y_valid[i],
                batch_size=self.batch_size,
                num_classes=self.num_classes
            ) for i in range(num_collaborators)
        ]
    
    def split_lognormal(self, labels, num_collaborators):
        from tqdm import trange
        labels = np.argmax(labels, axis=1)
        idx = [[np.nonzero(labels == (col + j) % self.num_classes)[0][np.arange(5) + (col // 10 * 10 + 5 * j)] \
            for j in range(2)] for col in range(num_collaborators)]
        idx = [np.hstack(tup) for tup in idx]
        assert all([len(i) == 10 for i in idx]), 'All collaborators should have 10 elements at this stage'
        props = np.random.lognormal(0, 2.0, (10,100,2))
        props = np.array([[[len(np.nonzero(labels==label)[0])-1000]] for label in range(10)])*props/np.sum(props,(1,2), keepdims=True)
        #idx = 1000*np.ones(10, dtype=np.int64)
        for user in trange(1000):
            for j in range(2):
                l = (user+j)%10
                num_samples = int(props[l,user//10,j])
                if np.count_nonzero(labels[np.hstack(idx)] == l) + num_samples < len(np.nonzero(labels==l)[0]):
                    idx_to_append = np.nonzero(labels == (user + j) % 10)[0][np.arange(num_samples) + np.count_nonzero(labels[np.hstack(idx)] == l)]
                    idx[user] = np.append(idx[user], idx_to_append)
        return idx

fl_data = UnbalancedFederatedDataset(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)


11501568/11490434 [==============================] - 1s 0us/step


In [7]:
from openfl.utilities.optimizers.keras import FedProxOptimizer

In [8]:
def build_model(input_shape,
                num_classes,
                **kwargs):
    """
    Define the model architecture.

    Args:
        input_shape (numpy.ndarray): The shape of the data
        num_classes (int): The number of classes of the dataset

    Returns:
        tensorflow.python.keras.engine.sequential.Sequential: The model defined in Keras

    """
    model = Sequential()
    
    model.add(tf.keras.Input(shape=input_shape))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=FedProxOptimizer(mu=1),
                  metrics=['accuracy'])

    return model   


In [9]:
#Create a federated model using the build model function and dataset
fl_model = FederatedModel(build_model, data_loader=fl_data)

In [10]:
collaborator_models = fl_model.setup(num_collaborators=1000)
 
collaborators = {f'col{col}':collaborator_models[col] for col in range(len(collaborator_models))}#, 'three':collaborator_models[2]}

100%|██████████| 1000/1000 [00:04<00:00, 204.90it/s]
<ipython-input-6-8e26411ae407>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array([self.X_train[idx] for idx in train_idx])
<ipython-input-6-8e26411ae407>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train = np.array([self.y_train[idx] for idx in train_idx])
100%|██████████| 1000/1000 [00:02<00:00, 409.22it/s]
<ipython-input-6-8e26411ae407>:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarray

In [11]:


#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')



Original training data size: 42000
Original validation data size: 18000

Collaborator one's training data size: 86
Collaborator one's validation data size: 10

Collaborator two's training data size: 12
Collaborator two's validation data size: 10



In [12]:
#Get the current values of the plan. Each of these can be overridden
import json
print(json.dumps(fx.get_plan(), indent=4, sort_keys=True))

"{\n    \"aggregator.settings.best_state_path\": \"save/keras_cnn_mnist_best.pbuf\",\n    \"aggregator.settings.db_store_rounds\": 2,\n    \"aggregator.settings.init_state_path\": \"save/keras_cnn_mnist_init.pbuf\",\n    \"aggregator.settings.last_state_path\": \"save/keras_cnn_mnist_last.pbuf\",\n    \"aggregator.settings.rounds_to_train\": 10,\n    \"aggregator.settings.write_logs\": true,\n    \"aggregator.template\": \"openfl.component.Aggregator\",\n    \"assigner.settings.task_groups.0.name\": \"train_and_validate\",\n    \"assigner.settings.task_groups.0.percentage\": 1.0,\n    \"assigner.settings.task_groups.0.tasks.0\": \"aggregated_model_validation\",\n    \"assigner.settings.task_groups.0.tasks.1\": \"train\",\n    \"assigner.settings.task_groups.0.tasks.2\": \"locally_tuned_model_validation\",\n    \"assigner.template\": \"openfl.component.RandomGroupedAssigner\",\n    \"collaborator.settings.db_store_rounds\": 1,\n    \"collaborator.settings.delta_updates\": false,\n    \"

In [ ]:

#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,override_config={'aggregator.settings.rounds_to_train':5, 'collaborator.settings.opt_treatment': 'CONTINUE_GLOBAL'})



/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1/1 [==============================] - 0s 46ms/step - loss: 2.4491 - accuracy: 0.2500


In [ ]:
#Save final model and load into keras
final_fl_model.save_native('final_model')
model = tf.keras.models.load_model('./final_model')


In [ ]:
#Test the final model on our test set
model.evaluate(test_images,test_labels)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(9,6), dpi=150)
plt.title('Keras MNIST unbalanced split')
plt.plot([0.07627802075538784, 0.07518334008473902, 0.09541350667830556, 0.13141966053564103, 0.15887578643299638], label='FedAvg')
plt.plot([0.07627802075538784, 0.07518334008473902, 0.09541350667830556, 0.1314459763141349, 0.15887578643299638], linestyle='--', label='FedProx (mu=1e-2)')
plt.plot([0.07627802075538784, 0.0751056043850258, 0.09555227747093886, 0.131649036151357, 0.15966261748969554], linestyle='--', label='FedProx (mu=1e-1)')
plt.plot([0.07627802075538784, 0.07517912408802659, 0.09641592293512076, 0.13676991989742965, 0.1684917744528502], linestyle='--', label='FedProx (mu=1e1)')

plt.legend()
plt.xticks(range(5))
plt.show()